# Purpose:
- Develop angle tuning analysis + touch responsive neurons
- Protocol:
    1. Z-score events
    2. touch response = (events during touch frames - during baseline) / num_touches
        - touch frames = touch + [0,1], before the answer lick
            - For miss trials - all the touches
        - baseline = before pole up
    3. ANOVA p < 0.05, at least one angle bin significantly different from 0, at least one pair of bins significantly different (Tukey's hsd).
    4. F-statistics of 10k shuffle, if 5% had F-stat > real F-stat, then not tuned.
- If touch response for any angle bin was significantly different from 0, then it is a touch responsive neuron
    - To be compared with GLM results

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

base_dir = Path(r'E:\TPM\JK\h5')

expert_mice_df = pd.read_csv(base_dir / 'expert_mice.csv', index_col=0)
use_mice_df = expert_mice_df.loc[expert_mice_df['depth_matched'].astype(bool) & 
                                 ~expert_mice_df['processing_error'].astype(bool) &
                                 ((expert_mice_df.session_type == 'training') |
                                  (expert_mice_df.session_type.str.contains('test')))]

In [2]:
use_mice_df.head()

,Unnamed: 0.1,mouse,plane,session,session_type,depth_matched,processing_error,remove_frame_ind
0,0,25,1,1,training,True,False,0
1,1,25,1,2,training,True,False,0
2,2,25,1,3,training,True,False,0
3,3,25,1,4,test_naive,True,False,0
4,4,25,1,5,training,True,False,0


In [67]:
# These helpfer functions copied to merged_df_annotation.py

def get_merged_df(ophys_frametime, behavior_frametime):
    # remove trials with errors
    refined_ophys_frametime = ophys_frametime.query('remove_trial==False')
    assert refined_ophys_frametime.remove_frame.values.sum() == 0

    # merge refined_ophys_frametime and behavior_frametime
    reduced_behavior_columns = np.setdiff1d(behavior_frametime.columns,
                                            np.setdiff1d(refined_ophys_frametime.columns,
                                                         ['trialNum', 'frame_index']))
    reduced_behavior_df = behavior_frametime[reduced_behavior_columns]
    merged_df = pd.merge(refined_ophys_frametime, reduced_behavior_df,
                         on=['trialNum', 'frame_index'], how='inner')
    return merged_df


def assign_pole_moving_frames(merged_df):
    # Assigne pole_moving_up and pole_moving_down to the frames
    # First check if all trials have correct pole up pole moving frames
    # Sometimes there is no pole_moving_frame
    # Just use -1 of the first pole up and +1 of pole up frame as pole_in_frame and pole_out_frame
    
    merged_df = merged_df.query('trial_type != "oo"').reset_index(drop=True)
    assert not merged_df.groupby('trialNum').apply(lambda x: len(np.where(x['pole_up_frame'] == 1)[0]) == 0).any()
    values_to_assign = merged_df.groupby('trialNum').apply(
        lambda x: x['frame_index'] == x['frame_index'].values[np.where(x['pole_up_frame'] == 1)[0][0] - 1]).reset_index(
        drop=True).values
    assert len(values_to_assign) == len(merged_df)
    merged_df['pole_in_frame'] = values_to_assign
    values_to_assign = merged_df.groupby('trialNum').apply(apply_pole_out).reset_index(
            drop=True).values
    assert len(values_to_assign) == len(merged_df)
    merged_df['pole_out_frame'] = values_to_assign

    return merged_df


def apply_pole_out(x):
     '''Apply pole_out_frame to the merged_df
     It is the last frame of pole_up_frame.
     It should be when pole out sound cue is on.
     If it stayed up till the end, then it is ambiguous, so don't apply it.
     '''
     if np.where(x['pole_up_frame']==True)[0][-1] < len(x)-1:
          return (x['frame_index'] == x['frame_index'].values[np.where(x['pole_up_frame']==1)[0][-1]]).reset_index(drop=True)
     else:
          return pd.Series([False]*len(x))


def assign_touch_response_frames(merged_df, post_touch_frames=1):
    # get touch response frames
    # add before_answer_touch_frame and after_answer_touch_frame
    touch_response_frames = merged_df.groupby('trialNum').apply(lambda x: _get_touch_response_frames(x,post_touch_frames=post_touch_frames))
    touch_response_frames = np.concatenate(touch_response_frames.values)
    values_to_assign = merged_df['frame_index'].isin(touch_response_frames).values
    assert len(values_to_assign) == len(merged_df)
    merged_df['touch_response_frame'] = values_to_assign
    merged_df['before_answer_touch_frame'] = False
    merged_df = merged_df.groupby('trialNum').apply(_get_before_answer_touch_frames)
    merged_df['after_answer_touch_frame'] = False
    merged_df = merged_df.groupby('trialNum').apply(_get_after_answer_touch_frames)
    return merged_df


def _get_touch_response_frames(x, post_touch_frames=1):
    # touch response frames = touch frames + 0,1,2 frames
    # per trial, to prevent touch rollover when pole was up till the end of the trial
    touch_frame_inds = np.where(x['touch_count']>0)[0]    
    touch_response_frame_inds = np.unique((touch_frame_inds[:,None] + np.arange(post_touch_frames+1)).flatten())
    touch_response_frame_inds = touch_response_frame_inds[touch_response_frame_inds < len(x)]
    touch_response_frames = x['frame_index'].values[touch_response_frame_inds]
    return touch_response_frames


def _get_before_answer_touch_frames(x):    
    if np.where(x['answer_lick_frame'])[0].size == 0:
        return x
    else:
        answer_lick_frame_ind = np.where(x['answer_lick_frame'])[0][0]
        x.iloc[:answer_lick_frame_ind]['before_answer_touch_frame'] = x.iloc[:answer_lick_frame_ind]['touch_response_frame']
    return x


def _get_after_answer_touch_frames(x):
    if np.where(x['answer_lick_frame'])[0].size == 0:
        return x
    else:
        answer_lick_frame_ind = np.where(x['answer_lick_frame'])[0][0]        
        x.iloc[answer_lick_frame_ind+1:]['after_answer_touch_frame'] = x.iloc[answer_lick_frame_ind+1:]['touch_response_frame']
    return x


# get normalized spikes
def get_normalized_spikes(roi_dir, ophys_frametime, merged_df, spk_norm='std'):
    spks = np.load(roi_dir / 'spks_reduced.npy')
    iscell = np.load(roi_dir / 'iscell.npy')
    cell_inds = np.where(iscell[:,0]==1)[0]
    spks = spks[cell_inds,:]
    assert spks.shape[1] == len(ophys_frametime)
    # deal with mismatched length
    if len(ophys_frametime) != len(merged_df):    
        removed_inds = np.where(ophys_frametime.frame_index.isin(merged_df.frame_index) == False)[0]
        # removed_tns = ophys_frametime.iloc[removed_inds].trialNum.unique()
        # print(f'JK{mouse:03} S{session:02} plane {plane} ophys_frametime and merged_df length mismatch:')
        # print(f'{len(removed_inds)} frames, {len(removed_tns)} trials')    
        spks = np.delete(spks, removed_inds, axis=1)
    assert spks.shape[1] == len(merged_df)

    # normalize spikes
    if spk_norm == 'std':
        norm_spks = (spks - spks.mean(axis=1)[:,np.newaxis]) / spks.std(axis=1)[:,np.newaxis]
    elif spk_norm == 'max':
        norm_spks = spks / spks.max(axis=1)[:,np.newaxis]
    elif spk_norm == 'none':
        norm_spks = spks
    else:
        raise ValueError('spk_norm should be either "std", "max", or "none"')
    norm_spks = xr.DataArray(norm_spks,
                             dims=('cell_index', 'frame_index'),
                             coords={'cell_index': cell_inds, 'frame_index': merged_df.frame_index.values})
    return norm_spks


def assign_baseline_frames(merged_df):
    def _get_baseline_frames(x):
        pole_in_frame_ind = np.where(x['pole_in_frame'])[0][0]
        x.iloc[:pole_in_frame_ind]['baseline_frame'] = True
        return x

    merged_df['baseline_frame'] = False
    merged_df = merged_df.groupby('trialNum').apply(_get_baseline_frames)
    return merged_df

In [8]:
mouse = 25
plane = 5
session_type = 'test_expert'
session = use_mice_df.query('mouse == @mouse and plane == @plane and session_type == @session_type').session.unique().astype(int)[0]

plane_dir = base_dir / f'{mouse:03}/plane_{plane}'
behavior_frametime = pd.read_pickle(plane_dir / f'JK{mouse:03}_S{session:02}_plane{plane}_frame_whisker_behavior.pkl')
roi_dir = plane_dir / f'{session:03}/plane0/roi'
ophys_frametime = pd.read_pickle(roi_dir / 'refined_frame_time.pkl')
merged_df = get_merged_df(ophys_frametime, behavior_frametime)
merged_df = assign_pole_moving_frames(merged_df)
merged_df = assign_baseline_frames(merged_df)
merged_df = assign_touch_response_frames(merged_df, post_touch_frames=1)
norm_spks = get_normalized_spikes(roi_dir, ophys_frametime, merged_df, spk_norm='std')
assert (norm_spks.frame_index.values == merged_df.frame_index.values).all()

In [15]:
behavior_frametime.columns

Index(['trialNum', 'frame_start_time', 'frame_end_time', 'trial_duration',
       'frame_index', 'kappaH_onset', 'kappaV_onset', 'phi_onset',
       'theta_onset', 'arc_length_onset', 'touch_count', 'delta_kappaH',
       'delta_kappaV', 'delta_phi', 'delta_theta', 'slide_distance',
       'touch_duration', 'pole_up_frame', 'pole_moving_frame', 'theta',
       'midpoint', 'amplitude', 'num_whisks', 'pole_angle', 'pole_ap_distance',
       'pole_radial_distance', 'num_lick_left', 'num_lick_right',
       'answer_lick_frame', 'answer_lick_left', 'answer_lick_right',
       'pre_answer_pole_up_frame', 'answer_period_frame',
       'post_answer_pole_up_frame', 'first_reward_lick_frame',
       'first_reward_lick_left', 'first_reward_lick_right', 'correct', 'wrong',
       'miss', 'trial_type', 'task_target', 'distractor', 'mouse_name',
       'session_name', 'session_type'],
      dtype='object')

In [4]:
merged_df.trialNum.unique()

array([12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 72, 73, 74, 75,
       76, 77, 78, 79, 80, 81, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101,
       112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 172, 173, 174, 175, 176, 177, 178, 179, 180,
       181, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 212, 213,
       214, 215, 216, 217, 218, 219, 220, 221, 232, 233, 234, 235, 236,
       237, 238, 239, 240, 241, 252, 253, 254, 255, 256, 257, 258, 259,
       260, 261, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 292,
       293, 294, 295, 296, 297, 298, 299, 300, 301, 312, 313, 314, 315,
       316, 317, 318, 319, 320, 321, 332, 333, 334, 335, 336, 337, 338,
       339, 340, 341, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361,
       372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 

In [14]:
merged_df.columns

Index(['trialNum', 'frame_start_time', 'frame_end_time', 'trial_duration',
       'frame_index', 'remove_frame', 'remove_trial', 'amplitude',
       'answer_lick_frame', 'answer_lick_left', 'answer_lick_right',
       'answer_period_frame', 'arc_length_onset', 'correct', 'delta_kappaH',
       'delta_kappaV', 'delta_phi', 'delta_theta', 'distractor',
       'first_reward_lick_frame', 'first_reward_lick_left',
       'first_reward_lick_right', 'kappaH_onset', 'kappaV_onset', 'midpoint',
       'miss', 'mouse_name', 'num_lick_left', 'num_lick_right', 'num_whisks',
       'phi_onset', 'pole_angle', 'pole_ap_distance', 'pole_moving_frame',
       'pole_radial_distance', 'pole_up_frame', 'post_answer_pole_up_frame',
       'pre_answer_pole_up_frame', 'session_name', 'session_type',
       'slide_distance', 'task_target', 'theta', 'theta_onset', 'touch_count',
       'touch_duration', 'trial_type', 'wrong', 'pole_in_frame',
       'pole_out_frame', 'baseline_frame', 'touch_response_frame',
 

In [16]:
merged_df[merged_df.trialNum==441][['frame_index','pole_in_frame', 'pole_moving_frame','pole_up_frame', 'pole_out_frame']]

,frame_index,pole_in_frame,pole_moving_frame,pole_up_frame,pole_out_frame
7713,8152,False,0.0,0.0,False
7714,8153,False,0.0,0.0,False
7715,8154,False,0.0,0.0,False
7716,8155,False,0.0,0.0,False
7717,8156,False,0.0,0.0,False
7718,8157,False,0.0,0.0,False
7719,8158,False,0.0,0.0,False
7720,8159,True,0.0,0.0,False
7721,8160,False,0.0,1.0,False
7722,8161,False,0.0,1.0,False


In [11]:
merged_df.groupby('trialNum').apply(lambda x: sum(x.pole_out_frame==True))

trialNum
12     1
13     1
14     1
15     1
16     1
      ..
437    1
438    1
439    1
440    1
441    1
Length: 220, dtype: int64

In [73]:
# These functions copied to touch_tuning.py

def get_touch_response_from_baseline(merged_df, norm_spks,
                                     touch_window='before_answer'):
    if 'touch_response_frame' not in merged_df.columns:
        merged_df = assign_touch_response_frames(merged_df)
    if 'baseline_frame' not in merged_df.columns:
        merged_df = assign_baseline_frames(merged_df)
    touch_trial_nums = merged_df[merged_df.touch_response_frame].trialNum.unique()
    touch_merged_df = merged_df[merged_df.trialNum.isin(touch_trial_nums)]
    touch_response_from_baseline = []
    for tn in touch_trial_nums:
        trial_df = touch_merged_df.query('trialNum == @tn')
        if touch_window == 'before_answer':
            touch_response_frame_inds = trial_df.query('before_answer_touch_frame').frame_index.values
        elif touch_window == 'after_answer':
            touch_response_frame_inds = trial_df.query('after_answer_touch_frame').frame_index.values
        elif touch_window == 'all':
            touch_response_frame_inds = trial_df.query('touch_response_frame').frame_index.values
        else:
            raise ValueError('touch_window should be either "before_answer", "after_answer", or "all"')
        baseline_frame_inds = trial_df.query('baseline_frame').frame_index.values
        touch_response_spks = norm_spks.sel(frame_index=touch_response_frame_inds).sum(dim='frame_index')
        baseline_spks = norm_spks.sel(frame_index=baseline_frame_inds).sum(dim='frame_index')

        temp_r_f_b = touch_response_spks - baseline_spks
        touch_response_from_baseline.append(temp_r_f_b.values)
    touch_response_from_baseline = np.stack(touch_response_from_baseline)
    touch_response_from_baseline = xr.DataArray(touch_response_from_baseline,
                                                dims=('trialNum', 'cell_index'),
                                                coords={'trialNum': touch_trial_nums.astype(int),
                                                        'cell_index': norm_spks.cell_index.values},
                                                attrs={'touch_window': touch_window})

    return touch_response_from_baseline


def get_touch_cell_inds(touch_response_xr, merged_df, pval_threshold=0.01):
    angles = np.sort(merged_df.pole_angle.unique())
    touch_trial_nums = merged_df.query('touch_response_frame').trialNum.unique()
    ttest_results = {}
    for cell_index in touch_response_xr.cell_index.values:
        cell_data = touch_response_xr.sel(cell_index=cell_index)
        ttest_results[cell_index] = {}
        for angle in angles:
            trials = merged_df.query('pole_angle == @angle').trialNum.values
            trials = np.intersect1d(trials, touch_trial_nums)
            cell_trials = cell_data.sel(trialNum=trials).values
            ttest_results[cell_index][angle] = ttest_ind(cell_trials, np.zeros_like(cell_trials))
    
    touch_cell_inds = []
    for cell_index, angles_data in ttest_results.items():
        for angle, ttest_result in angles_data.items():
            if ttest_result.pvalue < pval_threshold:
                touch_cell_inds.append(cell_index)
                break

    touch_cell_inds = np.array(touch_cell_inds)
    return touch_cell_inds, ttest_results


def get_tuned_cell_inds(touch_response_xr, merged_df, touch_cell_inds, pval_threshold=0.01):
    touch_trial_nums = merged_df[merged_df.touch_response_frame].trialNum.unique()
    angles = np.sort(merged_df.pole_angle.unique())
    if len(angles) > 2:
        # Perform ANOVA
        avova_results = {}
        angle_trials = []
        for angle in angles:
            trials = merged_df.query('pole_angle == @angle').trialNum.values
            trials = np.intersect1d(trials, touch_trial_nums)
            angle_trials.append(trials)
        for cell_index in touch_cell_inds:
            cell_data = touch_response_xr.sel(cell_index=cell_index)
            avova_results[cell_index] = f_oneway(*[cell_data.sel(trialNum=trials).values for trials in angle_trials])

        # Perform Tukey's HSD test
        tukey_results = {}
        trial_angles = [merged_df.query('trialNum == @tn').pole_angle.values[0] for tn in touch_trial_nums]
        assert len(trial_angles) == touch_response_xr.sizes['trialNum']
        tuned_cell_inds = []
        for cell_index in touch_cell_inds:
            if avova_results[cell_index].pvalue < pval_threshold:
                tukey_results[cell_index] = pairwise_tukeyhsd(touch_response_xr.sel(cell_index=cell_index),
                                                            trial_angles, alpha=0.05)
                if tukey_results[cell_index].reject.any():
                    tuned_cell_inds.append(cell_index)
        tuned_cell_inds = np.array(tuned_cell_inds)
        stat_results = {'anova': avova_results, 'tukey': tukey_results}

    elif len(angles) == 2:
        # Perform t-test
        ttest_results = {}
        angle_trials = []
        for angle in angles:
            trials = merged_df.query('pole_angle == @angle').trialNum.values
            trials = np.intersect1d(trials, touch_trial_nums)
            angle_trials.append(trials)
        tuned_cell_inds = []
        for cell_index in touch_cell_inds:
            cell_data = touch_response_xr.sel(cell_index=cell_index)
            ttest_results[cell_index] = ttest_ind(*[cell_data.sel(trialNum=trials).values for trials in angle_trials])
            if ttest_results[cell_index].pvalue < pval_threshold:
                tuned_cell_inds.append(cell_index)
        tuned_cell_inds = np.array(tuned_cell_inds)
        stat_results = {'ttest': ttest_results}
    else:
        raise ValueError('angles should have at least 2 unique values')

    return tuned_cell_inds, stat_results


In [153]:
touch_response_from_baseline = get_touch_response_from_baseline(merged_df, norm_spks,
                                                                touch_window='before_answer')
touch_cell_inds, ttest_results = get_touch_cell_inds(touch_response_from_baseline, merged_df, pval_threshold=0.01)
tuned_cell_inds, stat_results = get_tuned_cell_inds(touch_response_from_baseline, merged_df, touch_cell_inds, pval_threshold=0.01)

# Build codes for parallel processing
- save the results from before_answer, after_answer, and all
- for each plane
- then run stats (# of touch, tuned neurons, comparison among the touch windows)

In [70]:
def save_touch_tuning(mouse, plane, session):
    print(f'Processing JK{mouse:03} S{session:02} plane {plane}')
    plane_dir = base_dir / f'{mouse:03}/plane_{plane}'
    behavior_frametime = pd.read_pickle(plane_dir / f'JK{mouse:03}_S{session:02}_plane{plane}_frame_whisker_behavior.pkl')
    roi_dir = plane_dir / f'{session:03}/plane0/roi'

    touch_tuning_dir = roi_dir / 'touch_tuning'
    touch_tuning_dir.mkdir(exist_ok=True)

    touch_windows = ['before_answer', 'after_answer', 'all']

    ophys_frametime = pd.read_pickle(roi_dir / 'refined_frame_time.pkl')
    merged_df = get_merged_df(ophys_frametime, behavior_frametime)
    merged_df = assign_pole_moving_frames(merged_df)
    merged_df = assign_baseline_frames(merged_df)
    merged_df = assign_touch_response_frames(merged_df, post_touch_frames=1)
    norm_spks = get_normalized_spikes(roi_dir, ophys_frametime, merged_df, spk_norm='std')
    assert (norm_spks.frame_index.values == merged_df.frame_index.values).all()

    for touch_window in touch_windows:
        save_fn = touch_tuning_dir / f'touch_tuning_{touch_window}.npy'
        if save_fn.exists():
            continue
        touch_response_from_baseline = \
            get_touch_response_from_baseline(merged_df,
                            norm_spks, touch_window=touch_window)
        touch_cell_inds, ttest_results = \
            get_touch_cell_inds(touch_response_from_baseline,
                                             merged_df, pval_threshold=0.01)
        tuned_cell_inds, stat_results = \
            get_tuned_cell_inds(touch_response_from_baseline,
                            merged_df, touch_cell_inds, pval_threshold=0.01)
        results = {'touch_response': touch_response_from_baseline,
                   'touch_cell_inds': touch_cell_inds,
                   'tuned_cell_inds': tuned_cell_inds,
                   'ttest_results': ttest_results,                   
                   'stat_results': stat_results}
        np.save(save_fn, results)


In [ ]:

mouse = 25
plane = 1
session = 1
save_touch_tuning(mouse, plane, session)


In [13]:
# Check multiprocessing run
import os
mouse = 52
session = 6
plane = 7
plane_dir = base_dir / f'{mouse:03}/plane_{plane}'
roi_dir = plane_dir / f'{session:03}/plane0/roi'
touch_tuning_dir = roi_dir / 'touch_tuning'
fn = 'touch_tuning_all.npy'
os.path.exists(touch_tuning_dir / fn)

False

In [80]:
mouse = 52
session = 21
plane = 7
save_touch_tuning(mouse, plane, session)

Processing JK052 S21 plane 7


In [75]:
mouse = 52
session = 21
plane = 7

plane_dir = base_dir / f'{mouse:03}/plane_{plane}'
behavior_frametime = pd.read_pickle(plane_dir / f'JK{mouse:03}_S{session:02}_plane{plane}_frame_whisker_behavior.pkl')
roi_dir = plane_dir / f'{session:03}/plane0/roi'

touch_tuning_dir = roi_dir / 'touch_tuning'
touch_tuning_dir.mkdir(exist_ok=True)

touch_windows = ['before_answer', 'after_answer', 'all']

ophys_frametime = pd.read_pickle(roi_dir / 'refined_frame_time.pkl')
merged_df = get_merged_df(ophys_frametime, behavior_frametime)
merged_df = assign_pole_moving_frames(merged_df)
# merged_df = assign_baseline_frames(merged_df)

In [77]:
post_touch_frames = 1
touch_response_frames = merged_df.groupby('trialNum').apply(lambda x: _get_touch_response_frames(x,post_touch_frames=post_touch_frames))
touch_response_frames = np.concatenate(touch_response_frames.values)


In [81]:
values_to_assign = merged_df['frame_index'].isin(touch_response_frames).values
assert len(values_to_assign) == len(merged_df)
merged_df['touch_response_frame'] = values_to_assign
merged_df['before_answer_touch_frame'] = False

In [88]:
temp_df = merged_df.groupby('trialNum').apply(_get_before_answer_touch_frames)
trial_nums = temp_df.trialNum.unique()
trialNum = trial_nums[-10]
temp_df.query('trialNum==@trialNum')[['touch_response_frame', 'answer_lick_frame',
                                      'before_answer_touch_frame', 'pole_up_frame']]

,touch_response_frame,answer_lick_frame,before_answer_touch_frame,pole_up_frame
6270,False,False,False,0.0
6271,False,False,False,0.0
6272,False,False,False,0.0
6273,False,False,False,0.0
6274,False,False,False,0.0
6275,False,False,False,0.0
6276,False,False,False,0.0
6277,False,False,False,0.0
6278,False,False,False,0.0
6279,False,False,False,1.0
